# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 14 2022 1:46PM,239584,21,617811,"NBTY Global, Inc.",Released
1,Jun 14 2022 1:35PM,239583,19,ACG-2102490329,ACG North America LLC,Released
2,Jun 14 2022 1:35PM,239583,19,ACG-2102490330,ACG North America LLC,Released
3,Jun 14 2022 1:32PM,239582,10,Eme-100410,Emerginnova,Released
4,Jun 14 2022 1:32PM,239582,10,Enova-10135,Emerginnova,Released
5,Jun 14 2022 1:32PM,239582,10,Enova-10136,Emerginnova,Released
6,Jun 14 2022 1:31PM,239581,12,HH-35120,Hush Hush,Executing
7,Jun 14 2022 1:31PM,239581,12,HH-35121,Hush Hush,Released
8,Jun 14 2022 1:31PM,239581,12,HH-35122,Hush Hush,Released
9,Jun 14 2022 1:30PM,239580,18,CLRSPC-2875417,"ClearSpec, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
58,239581,Executing,1
59,239581,Released,2
60,239582,Released,3
61,239583,Released,2
62,239584,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
239580,NaN,NaN,1.0
239581,NaN,1.0,2.0
239582,NaN,NaN,3.0
239583,NaN,NaN,2.0
239584,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239446,15.0,21.0,118.0
239479,0.0,0.0,33.0
239481,0.0,0.0,1.0
239482,0.0,1.0,0.0
239485,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239446,15,21,118
239479,0,0,33
239481,0,0,1
239482,0,1,0
239485,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239446,15,21,118
1,239479,0,0,33
2,239481,0,0,1
3,239482,0,1,0
4,239485,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239446,15,21,118
1,239479,,,33
2,239481,,,1
3,239482,,1,
4,239485,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 14 2022 1:46PM,239584,21,"NBTY Global, Inc."
1,Jun 14 2022 1:35PM,239583,19,ACG North America LLC
3,Jun 14 2022 1:32PM,239582,10,Emerginnova
6,Jun 14 2022 1:31PM,239581,12,Hush Hush
9,Jun 14 2022 1:30PM,239580,18,"ClearSpec, LLC"
10,Jun 14 2022 1:25PM,239579,19,ACG North America LLC
14,Jun 14 2022 1:24PM,239578,18,"ClearSpec, LLC"
15,Jun 14 2022 1:16PM,239575,10,"Methapharm, Inc."
44,Jun 14 2022 1:16PM,239551,20,"ACI Healthcare USA, Inc."
72,Jun 14 2022 1:14PM,239577,10,"Methapharm, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 14 2022 1:46PM,239584,21,"NBTY Global, Inc.",,,1
1,Jun 14 2022 1:35PM,239583,19,ACG North America LLC,,,2
2,Jun 14 2022 1:32PM,239582,10,Emerginnova,,,3
3,Jun 14 2022 1:31PM,239581,12,Hush Hush,,1,2
4,Jun 14 2022 1:30PM,239580,18,"ClearSpec, LLC",,,1
5,Jun 14 2022 1:25PM,239579,19,ACG North America LLC,,,4
6,Jun 14 2022 1:24PM,239578,18,"ClearSpec, LLC",,,1
7,Jun 14 2022 1:16PM,239575,10,"Methapharm, Inc.",,,29
8,Jun 14 2022 1:16PM,239551,20,"ACI Healthcare USA, Inc.",,8,20
9,Jun 14 2022 1:14PM,239577,10,"Methapharm, Inc.",,,6


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 14 2022 1:46PM,239584,21,"NBTY Global, Inc.",1,,
1,Jun 14 2022 1:35PM,239583,19,ACG North America LLC,2,,
2,Jun 14 2022 1:32PM,239582,10,Emerginnova,3,,
3,Jun 14 2022 1:31PM,239581,12,Hush Hush,2,1,
4,Jun 14 2022 1:30PM,239580,18,"ClearSpec, LLC",1,,
5,Jun 14 2022 1:25PM,239579,19,ACG North America LLC,4,,
6,Jun 14 2022 1:24PM,239578,18,"ClearSpec, LLC",1,,
7,Jun 14 2022 1:16PM,239575,10,"Methapharm, Inc.",29,,
8,Jun 14 2022 1:16PM,239551,20,"ACI Healthcare USA, Inc.",20,8,
9,Jun 14 2022 1:14PM,239577,10,"Methapharm, Inc.",6,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 14 2022 1:46PM,239584,21,"NBTY Global, Inc.",1,,
1,Jun 14 2022 1:35PM,239583,19,ACG North America LLC,2,,
2,Jun 14 2022 1:32PM,239582,10,Emerginnova,3,,
3,Jun 14 2022 1:31PM,239581,12,Hush Hush,2,1,
4,Jun 14 2022 1:30PM,239580,18,"ClearSpec, LLC",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 14 2022 1:46PM,239584,21,"NBTY Global, Inc.",1.0,NaN,NaN
1,Jun 14 2022 1:35PM,239583,19,ACG North America LLC,2.0,NaN,NaN
2,Jun 14 2022 1:32PM,239582,10,Emerginnova,3.0,NaN,NaN
3,Jun 14 2022 1:31PM,239581,12,Hush Hush,2.0,1.0,NaN
4,Jun 14 2022 1:30PM,239580,18,"ClearSpec, LLC",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 14 2022 1:46PM,239584,21,"NBTY Global, Inc.",1.0,0.0,0.0
1,Jun 14 2022 1:35PM,239583,19,ACG North America LLC,2.0,0.0,0.0
2,Jun 14 2022 1:32PM,239582,10,Emerginnova,3.0,0.0,0.0
3,Jun 14 2022 1:31PM,239581,12,Hush Hush,2.0,1.0,0.0
4,Jun 14 2022 1:30PM,239580,18,"ClearSpec, LLC",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,6707289,300.0,41.0,5.0
12,479125,2.0,3.0,0.0
15,958043,158.0,42.0,20.0
18,958287,4.0,0.0,0.0
19,1676990,11.0,0.0,0.0
20,479086,39.0,19.0,0.0
21,958032,3.0,1.0,0.0
22,239497,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,6707289,300.0,41.0,5.0
1,12,479125,2.0,3.0,0.0
2,15,958043,158.0,42.0,20.0
3,18,958287,4.0,0.0,0.0
4,19,1676990,11.0,0.0,0.0
5,20,479086,39.0,19.0,0.0
6,21,958032,3.0,1.0,0.0
7,22,239497,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,300.0,41.0,5.0
1,12,2.0,3.0,0.0
2,15,158.0,42.0,20.0
3,18,4.0,0.0,0.0
4,19,11.0,0.0,0.0
5,20,39.0,19.0,0.0
6,21,3.0,1.0,0.0
7,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,300.0
1,12,Released,2.0
2,15,Released,158.0
3,18,Released,4.0
4,19,Released,11.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,18,19,20,21,22
Status,,,,,,,,
Completed,5.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0
Executing,41.0,3.0,42.0,0.0,0.0,19.0,1.0,0.0
Released,300.0,2.0,158.0,4.0,11.0,39.0,3.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,18,19,20,21,22
0,Completed,5.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0
1,Executing,41.0,3.0,42.0,0.0,0.0,19.0,1.0,0.0
2,Released,300.0,2.0,158.0,4.0,11.0,39.0,3.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,18,19,20,21,22
0,Completed,5.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0
1,Executing,41.0,3.0,42.0,0.0,0.0,19.0,1.0,0.0
2,Released,300.0,2.0,158.0,4.0,11.0,39.0,3.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()